In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import creep_event_picker as cep
import obspy
import matplotlib.dates as mdates
import cmcrameri.cm as cmc
import math
%matplotlib qt


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
def check_dir(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path, exist_ok=True)

Import creepmeter database

In [4]:
Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Picked'] != 'YES'].index,inplace=True)
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Manual_check_initial'] == 'N/A'].index,inplace=True)
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe["Checked for missing events"] != 'YES'].index,inplace=True)
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe["Checked start picks"] == 'YES'].index,inplace=True)
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe["Checked end picks"] != 'NO'].index,inplace=True)

#Creepmeter_dataframe.reset_index(inplace=True)
#Creepmeter_dataframe.drop(['Index'],axis=1,inplace=True)
Creepmeter_dataframe

,Network,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude,Length,Depth,Obliquity,Creepmeter_full_name
0,EAF,BAL1,BAL1_1T,2023-03-12T10:47:55.000000Z,2023-09-05T13:14:55.000000Z,0.016667,1T,37.9903,38.1990,5 m,NaN,30,Balikburnu
1,PARK,C461,C461_10T,2004-09-29T02:20:00.000000Z,2022-06-20T12:40:00.000000Z,0.001667,10T,35.7240,-120.2820,NaN,NaN,30,Highway 46
2,PARK,C462,C462_1T,2021-08-19T19:10:11.000000Z,2024-03-22T01:05:11.000000Z,0.016667,1T,35.7240,-120.2820,NaN,NaN,30,Highway 46
3,UTA,CAN1,CAN1_30T,2012-06-12T15:33:00.000000Z,2017-06-15T08:33:00.000000Z,0.000556,30T,38.0198,-110.0374,7.5 m,NaN,45,Canyonlands
4,NAF,CER1,CER1_30S,2019-10-09T06:46:00.000000Z,2024-05-23T10:25:30.000000Z,0.033333,30S,40.8947,32.7773,20 m,NaN,29,Cerkes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,HOL,XSJ3,XSJ3_10T,2007-02-15T20:19:59.000000Z,2016-12-08T18:09:59.000000Z,0.001667,10T,36.8360,-121.5210,20 m,NaN,30,San Juan Bautista #3
121,PARK,XTA1,XTA1_10T,1985-12-18T00:00:00.000000Z,2024-10-09T18:50:00.000000Z,0.001667,10T,35.8900,-120.4270,30 m,1 m,30,Taylor Ranch
122,PARK,XVA1,XVA1_10T,1987-05-29T00:00:00.000000Z,2017-10-23T22:10:00.000000Z,0.001667,10T,35.9220,-120.4620,30 m,1 m,30,Varian
123,EAF,YZE1,YZE1_1T,2023-09-07T10:36:07.000000Z,2024-05-26T12:59:07.000000Z,0.016667,1T,38.1791,38.7526,20 m,NaN,30,Yazica East


In [5]:
cutter = 2
n=10
cmap = cmc.roma #select the colormap

slicedCM = cmap(np.linspace(0, 1, cutter+1)) #list of colors that make up the colormap, can be called by using slicedCM[j] where j is the loop number in the fore loop

In [6]:
frequency_dict = {
        1/60: '1S',    # 1 second
        1/6: '10S',  # 10 seconds
        1/2: '30S',  # 30 seconds
        1: '1T',  # 1 minute
        2: '2T',  # 2 minutes
        5: '5T',  # 5 minutes
        10: '10T',# 10 minutes
        15: '15T', # 15 minutes
        30: '30T',# 30 minutes
        60: '60T'    # 60 minutes
    }

In [11]:
plt.close('all')
for i in range(len(Creepmeter_dataframe)):
    file_code = Creepmeter_dataframe.File_code.iloc[i]
    path = "../../Data/DATA_tidied/SAC/{q}.SAC".format(q =file_code)
    path_save = "../../Data/DATA_tidied/Picks/"
    check_dir(path_save)
    print(path)
    try:
        df_picks = pd.read_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =file_code),index_col=0)
        column_names_list = df_picks.columns.tolist()
        if 'Quality_check_new' not in column_names_list:
            print(file_code,'no QC')
        
        
            st = obspy.read(path)
            tr = st[0]
            #tr.interpolate(sampling_rate=tr.stats.sampling_rate)
            tm = tr.times("utcdatetime")
            creep = tr.data
            sample_rate =  round(tr.stats.delta)/60
            tm_dt = []
            for w in range(len(tm)):
                tm_new = tm[w].datetime
                tm_dt.append(tm_new)
            print('now interpolate')
            tm_int = np.array(tm_dt)
            creep_int = creep
            #tm_int, creep_int, upsampled = cep.interpolate(tm_dt,creep,sample_rate,5)

        
            df_picks.sort_values(by='ST',inplace=True)
            try:
                df_picks.reset_index(inplace=True,drop=True)
            except:
                dummy=10
            
        
            df_end=pd.DataFrame(columns=['ET'])
            df_start=pd.DataFrame(columns=['ST'])
            QC = []
            for p in range(math.ceil(len(df_picks)/cutter)):
                fig=plt.figure(figsize=(20,10))
                fig.tight_layout()
                ax = plt.subplot(1,1,1)

                df_selecta = df_picks.copy(deep=True)
                boolarr = np.logical_and(df_selecta.index >= cutter*(p),df_selecta.index < cutter*(p+1))
                df_selecta = df_selecta[boolarr]
                #print(df_selecta)
                fig.suptitle('events {a} - {b} of {c}'.format(a = df_selecta.index[0]+1,b= df_selecta.index[-1]+1,c =len(df_picks) ))
                boolarr_selecta = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_selecta.ST.iloc[0]).replace(tzinfo=None)\
                                                -dt.timedelta(days=10),\
                                                        np.array(tm_int)<=pd.to_datetime(df_selecta.ET.iloc[-1]).replace(tzinfo=None)\
                                                            +dt.timedelta(days = 30))
                tm_selecta = tm_int[boolarr_selecta]
                creep_selecta = creep_int[boolarr_selecta]
                plt.plot(tm_selecta,creep_selecta,marker='o', color='k', linestyle='-', linewidth=2, markersize=6)

                for j in range(len(df_picks)):
                    boolarr = np.logical_and(np.array(tm_selecta)>=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None),\
                                                    np.array(tm_selecta)<=pd.to_datetime(df_picks.ET.iloc[j]).replace(tzinfo=None))
                    event_creep = creep_selecta[boolarr]
                    event_time = tm_selecta[boolarr]
                    #print(event_time)
                    ax.plot(event_time,event_creep,color='orange',linewidth=3)
                    try:
                        ax.text(event_time[0],event_creep[0],'{k}/{n}'.format(k=df_picks.index[j]+1,n=len(df_picks)),color='green')
                        ax.text(event_time[-1],event_creep[-1],'{k}/{n}'.format(k=df_picks.index[j]+1,n=len(df_picks)),color='red')
                    except IndexError:
                        dummy=10

                ax.set_xlim([tm_selecta[0],tm_selecta[-1]])
                ax.set_ylim([np.nanmin(np.isfinite(creep_selecta) * creep_selecta), 
                    np.nanmax(np.isfinite(creep_selecta) * creep_selecta)])
                
                for j in range(len(df_selecta)):
                    if 'Quality_check' not in df_selecta.columns.to_list():
                        color='blue'
                    elif df_selecta["Quality_check"].iloc[j]=="CE":
                        color='green'
                    elif df_selecta["Quality_check"].iloc[j]=="NCE":
                        color="red"

                    boolarr = np.logical_and(np.array(tm_selecta)>=pd.to_datetime(df_selecta.ST.iloc[j]).replace(tzinfo=None),\
                                                    np.array(tm_selecta)<=pd.to_datetime(df_selecta.ET.iloc[j]).replace(tzinfo=None))
                    event_creep = creep_selecta[boolarr]
                    event_time = tm_selecta[boolarr]
                    #print(event_time)
                    ax.plot(event_time,event_creep,color=color,linewidth=j+1,zorder=100-(j*20))
                    ax.scatter([event_time[0]],[event_creep[0]],c='green',marker='s',s=20)
                    ax.scatter([event_time[-1]],[event_creep[-1]],c='red',zorder=10,s=10)
                    ax.text(event_time[0],event_creep[0],'{k}/{n}'.format(k=df_selecta.index[j]+1,n=len(df_picks)),color='green')
                    ax.text(event_time[-1],event_creep[-1],'{k}/{n}'.format(k=df_selecta.index[j]+1,n=len(df_picks)),color='red')

                
                ##### define start, end and QC ####
                
                
                scale = 1.1
                zp = cep.ZoomPan()
                figZoom = zp.zoom_factory(ax, base_scale = scale)
                figPan = zp.pan_factory(ax)
                s = 1
                fig.set_size_inches(11,11)
                fig.tight_layout()
                plt.show()
                for m in range(len(df_selecta)):
                    print('{m}/{n}'.format(m=df_selecta.index[m]+1,n=len(df_picks)))

                    print('is the start pick good?\n esc=yes, click=no')
                    check_start = plt.ginput(1,timeout = -1)
                    if len(check_start) ==1:
                        for sublist in check_start:
                            df = pd.DataFrame({'ST':[sublist[0]]})
                            df_start = pd.concat([df_start,df])
                        print('new start picked\n {k}'.format(k=mdates.num2date(sublist[0])))
                    else:
                        print('start all good! :D')
                        df = pd.DataFrame({'ST':mdates.date2num([df_selecta.ST.loc[df_selecta.index[m]]])})
                        df_start = pd.concat([df_start,df])

                    print("now pick the end\n esc=keep current, click=pick new")
                    pts = plt.ginput(1,timeout = -1)
                    if len(pts)==1:
                        for sublist in pts:
                            df = pd.DataFrame({'ET':[sublist[0]]})
                            df_end = pd.concat([df_end,df])
                            print('new end picked\n {k}'.format(k=mdates.num2date(sublist[0])))
                    else:
                        print('end all good! :D')
                        df = pd.DataFrame({'ET':mdates.date2num([df_selecta.ET.loc[df_selecta.index[m]]])})
                        df_end = pd.concat([df_end,df])
                    
                    
                    print('is this a creep event?\n esc=no, click=yes')
                    check_pts = plt.ginput(1,timeout = -1)
                    
                    if len(check_pts) == 1:
                        QC.append('CE')
                        print('YES!! :D')
                    else:
                        QC.append('NCE')
                        print('Nope :(')

                plt.close()
            rounder = frequency_dict[sample_rate]
            for r in range(len(df_end)):
                time_convert = mdates.num2date(float(df_end.ET.iloc[r]))
        
                s3 = pd.Series(time_convert).dt.round(rounder)
                df_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)  

            
            for r in range(len(df_start)):
                time_convert = mdates.num2date(float(df_start.ST.iloc[r]))
                s3 = pd.Series(time_convert).dt.round(rounder)
                df_start.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None) 
            
            #df_picks_end.ET= mdates.num2date(df_picks_end.ET)
            df_end.reset_index(inplace=True)
            df_start.reset_index(inplace=True)
            df_end.drop(columns=['index'],inplace=True)
            df_start.drop(columns=['index'],inplace=True)
            
            df_picks['ST_new'] = df_start['ST']
            df_picks['ET_new'] = df_end['ET']
            df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
            df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
            df_picks['ST_new'] = df_picks['ST_new'].astype(str).str[0:19]
            df_picks['ET_new'] = df_picks['ET_new'].astype(str).str[0:19]
            df_picks['Quality_check_new'] = QC
            df_picks.to_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =file_code))
        else:
            print(file_code,'QC done')
    except FileNotFoundError:
        print(file_code,'NO EVENTS')

../../Data/DATA_tidied/SAC/BAL1_1T.SAC
BAL1_1T QC done
../../Data/DATA_tidied/SAC/C461_10T.SAC
C461_10T QC done
../../Data/DATA_tidied/SAC/C462_1T.SAC
C462_1T QC done
../../Data/DATA_tidied/SAC/CAN1_30T.SAC
CAN1_30T QC done
../../Data/DATA_tidied/SAC/CER1_30S.SAC
CER1_30S QC done
../../Data/DATA_tidied/SAC/CER1_1T.SAC
CER1_1T QC done
../../Data/DATA_tidied/SAC/CFW1_1T.SAC
CFW1_1T QC done
../../Data/DATA_tidied/SAC/CFW1_10T.SAC
CFW1_10T QC done
../../Data/DATA_tidied/SAC/CFW1_60T.SAC
CFW1_60T QC done
../../Data/DATA_tidied/SAC/CHA1_10T.SAC
CHA1_10T QC done
../../Data/DATA_tidied/SAC/CHE1_30S.SAC
CHE1_30S QC done
../../Data/DATA_tidied/SAC/CHE1_1T.SAC
CHE1_1T QC done
../../Data/DATA_tidied/SAC/CHE1_5T.SAC
CHE1_5T QC done
../../Data/DATA_tidied/SAC/CHP1_1T.SAC
CHP1_1T NO EVENTS
../../Data/DATA_tidied/SAC/CHP1_10T.SAC
CHP1_10T NO EVENTS
../../Data/DATA_tidied/SAC/COZ1_1T.SAC
COZ1_1T QC done
../../Data/DATA_tidied/SAC/COZ1_10T.SAC
COZ1_10T QC done
../../Data/DATA_tidied/SAC/COZ1_60T.SAC
COZ

/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_9274/3352319848.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_start = pd.concat([df_start,df])


new start picked
 2023-04-06 07:04:57.089424+00:00
now pick the end
 esc=keep current, click=pick new


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_9274/3352319848.py:127: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_end = pd.concat([df_end,df])


new end picked
 2023-04-06 17:50:19.919051+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
2/18
is the start pick good?
 esc=yes, click=no
new start picked
 2023-04-07 03:59:57.804823+00:00
now pick the end
 esc=keep current, click=pick new
new end picked
 2023-04-07 09:16:28.723857+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
3/18
is the start pick good?
 esc=yes, click=no
new start picked
 2023-04-10 12:02:59.508728+00:00
now pick the end
 esc=keep current, click=pick new
end all good! :D
is this a creep event?
 esc=no, click=yes
YES!! :D
4/18
is the start pick good?
 esc=yes, click=no
new start picked
 2023-04-10 23:18:00.450884+00:00
now pick the end
 esc=keep current, click=pick new
end all good! :D
is this a creep event?
 esc=no, click=yes
YES!! :D
5/18
is the start pick good?
 esc=yes, click=no
start all good! :D
now pick the end
 esc=keep current, click=pick new
end all good! :D
is this a creep event?
 esc=no, click=yes
Nope :(
6/18
is the start pick good?


...

/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_9274/3352319848.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_start = pd.concat([df_start,df])


new start picked
 2023-04-01 12:26:03.089628+00:00
now pick the end
 esc=keep current, click=pick new


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_9274/3352319848.py:127: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_end = pd.concat([df_end,df])


new end picked
 2023-05-23 06:00:39.040350+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
2/5
is the start pick good?
 esc=yes, click=no
start all good! :D
now pick the end
 esc=keep current, click=pick new
end all good! :D
is this a creep event?
 esc=no, click=yes
Nope :(
3/5
is the start pick good?
 esc=yes, click=no
start all good! :D
now pick the end
 esc=keep current, click=pick new
end all good! :D
is this a creep event?
 esc=no, click=yes


KeyboardInterrupt: 

split 1 at 2021-08-20 02:49:11

'30S'

In [ ]:
time_convert = mdates.num2date(float(df_start.ST.iloc[0]))

In [ ]:
time_convert

datetime.datetime(2019, 10, 18, 14, 31, 54, 750887, tzinfo=datetime.timezone.utc)

In [ ]:
s3 = pd.Series(time_convert).dt.round("30S")
s3

0   2019-10-18 14:32:00+00:00
dtype: datetime64[ns, UTC]

In [ ]:
st[0].stats

In [ ]:
%matplotlib inline
tr = st[0]
tr.plot();

In [ ]:
df_selecta

In [ ]:
plt.close('all')
for i in range(16,18):#len(Creepmeter_dataframe)):
    path = "../../../Data/{p}/SAC/{q}.SAC".format(p=Creepmeter_dataframe.Network.iloc[i],q =Creepmeter_dataframe.File_code.iloc[i])
    path_save = "../../../Data/{p}/Picks/".format(p=Creepmeter_dataframe.Network.iloc[i])
    check_dir(path_save)
    print(path)
    st = obspy.read(path)
    tr = st[0]
    tr.interpolate(sampling_rate=tr.stats.sampling_rate)
    tm = tr.times("utcdatetime")
    creep = tr.data
    sample_rate =  round(tr.stats.delta)/60
    tm_dt = []
    for w in range(len(tm)):
        tm_new = tm[w].datetime
        tm_dt.append(tm_new)
    print('now interpolate')
    tm_int, creep_int, upsampled = cep.interpolate(tm_dt,creep,sample_rate,5)
    


    
    #plt.plot(tm_int,creep_int)

    df_picks = pd.read_csv("../../../Data/{p}/Picks/{q}_picks.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
                                                                          q =Creepmeter_dataframe.File_code.iloc[i]),index_col=0)
    df_picks.sort_values(by='ST_new',inplace=True)
    try:
        df_picks.reset_index(inplace=True)
    except:
        dummy=10
    
   
    df_end=pd.DataFrame(columns=['ET_new'])
    df_start=pd.DataFrame(columns=['ST_new'])
    QC = []
    for p in range(math.ceil(len(df_picks)/cutter)):
        fig = plt.figure(figsize=(7,6))
        ax = plt.subplot(1,1,1)

        df_selecta = df_picks.copy(deep=True)
        boolarr = np.logical_and(df_selecta.index >= cutter*(p),df_selecta.index < cutter*(p+1))
        df_selecta = df_selecta[boolarr]
        #print(df_selecta)
        fig.suptitle('events {a} - {b} of {c}'.format(a = df_selecta.index[0]+1,b= df_selecta.index[-1]+1,c =len(df_picks) ))
        boolarr_selecta = np.logical_and(np.array(tm_int)>=pd.to_datetime(df_selecta.ST.iloc[0]).replace(tzinfo=None)\
                                         -dt.timedelta(days=10),\
                                                np.array(tm_int)<=pd.to_datetime(df_selecta.ET.iloc[-1]).replace(tzinfo=None)\
                                                    +dt.timedelta(days = 30))
        tm_selecta = tm_int[boolarr_selecta]
        creep_selecta = creep_int[boolarr_selecta]
        plt.plot(tm_selecta,creep_selecta,color='k')

        for j in range(len(df_picks)):
            boolarr = np.logical_and(np.array(tm_selecta)>=pd.to_datetime(df_picks.ST_new.iloc[j]).replace(tzinfo=None),\
                                            np.array(tm_selecta)<=pd.to_datetime(df_picks.ET_new.iloc[j]).replace(tzinfo=None))
            event_creep = creep_selecta[boolarr]
            event_time = tm_selecta[boolarr]
            #print(event_time)
            ax.plot(event_time,event_creep,color='orange',linewidth=3)
            try:
                ax.text(event_time[0],event_creep[0],'{k}/{n}'.format(k=df_picks.index[j]+1,n=len(df_picks)),color='green')
                ax.text(event_time[-1],event_creep[-1],'{k}/{n}'.format(k=df_picks.index[j]+1,n=len(df_picks)),color='red')
            except IndexError:
                dummy=10

        ax.set_xlim([tm_selecta[0],tm_selecta[-1]])
        ax.set_ylim([np.min(creep_selecta),np.max(creep_selecta)])
        for j in range(len(df_selecta)):
            if df_selecta["Quality_check_new"].iloc[j]=="CE":
                color='green'
            else:
                color="red"
            boolarr = np.logical_and(np.array(tm_selecta)>=pd.to_datetime(df_selecta.ST_new.iloc[j]).replace(tzinfo=None),\
                                            np.array(tm_selecta)<=pd.to_datetime(df_selecta.ET_new.iloc[j]).replace(tzinfo=None))
            event_creep = creep_selecta[boolarr]
            event_time = tm_selecta[boolarr]
            #print(event_time)
            ax.plot(event_time,event_creep,color=color,linewidth=j+1,zorder=100-(j*20))
            try:
                ax.scatter([event_time[0]],[event_creep[0]],c='green',marker='s',s=20)
                ax.scatter([event_time[-1]],[event_creep[-1]],c='red',zorder=10,s=10)
                ax.text(event_time[0],event_creep[0],'{k}/{n}'.format(k=df_selecta.index[j]+1,n=len(df_picks)),color='green')
                ax.text(event_time[-1],event_creep[-1],'{k}/{n}'.format(k=df_selecta.index[j]+1,n=len(df_picks)),color='red')
            except IndexError:
                dummy=10

        
        ##### define start, end and QC ####
        
        
        scale = 1.1
        zp = cep.ZoomPan()
        figZoom = zp.zoom_factory(ax, base_scale = scale)
        figPan = zp.pan_factory(ax)
        s = 1
        fig.set_size_inches(11,11)
        fig.tight_layout()
        plt.show()
        for m in range(len(df_selecta)):
            print('{m}/{n}'.format(m=df_selecta.index[m]+1,n=len(df_picks)))

            print('is the start pick good?\n esc=yes, click=no')
            check_start = plt.ginput(1,timeout = -1)
            if len(check_start) ==1:
                for sublist in check_start:
                    df = pd.DataFrame({'ST':[sublist[0]]})
                    df_start = pd.concat([df_start,df])
                print('new start picked\n {k}'.format(k=mdates.num2date(sublist[0])))
            else:
                print('start all good! :D')
                df = pd.DataFrame({'ST':mdates.date2num([df_selecta.ST.loc[df_selecta.index[m]]])})
                df_start = pd.concat([df_start,df])

            print("now pick the end\n esc=keep current, click=pick new")
            pts = plt.ginput(1,timeout = -1)
            if len(pts)==1:
                for sublist in pts:
                    df = pd.DataFrame({'ET':[sublist[0]]})
                    df_end = pd.concat([df_end,df])
                    print('new end picked\n {k}'.format(k=mdates.num2date(sublist[0])))
            else:
                print('end all good! :D')
                df = pd.DataFrame({'ET':mdates.date2num([df_selecta.ET.loc[df_selecta.index[m]]])})
                df_end = pd.concat([df_end,df])
            
            
            print('is this a creep event?\n esc=no, click=yes')
            check_pts = plt.ginput(1,timeout = -1)
            
            if len(check_pts) == 1:
                QC.append('CE')
                print('YES!! :D')
            else:
                QC.append('NCE')
                print('Nope :(')

        plt.close()
    for r in range(len(df_end)):
        time_convert = mdates.num2date(float(df_end.ET.iloc[r]))
        s3 = pd.Series(time_convert).dt.round("1min")
        df_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)  

    
    for r in range(len(df_start)):
        time_convert = mdates.num2date(float(df_start.ST.iloc[r]))
        s3 = pd.Series(time_convert).dt.round("1min")
        df_start.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None) 
    
    #df_picks_end.ET= mdates.num2date(df_picks_end.ET)
    df_end.reset_index(inplace=True)
    df_start.reset_index(inplace=True)
    df_end.drop(columns=['index'],inplace=True)
    df_start.drop(columns=['index'],inplace=True)
    
    df_picks['ET_new'] = df_end['ET']
    df_picks['ST_new'] = df_start['ST']
    df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
    df_picks['ET'] = df_picks['ET'].astype(str).str[0:19]
    df_picks['ET_new'] = df_picks['ET_new'].astype(str).str[0:19]
    df_picks['ST_new'] = df_picks['ST_new'].astype(str).str[0:19]
    df_picks['Quality_check_new'] = QC
    print(df_picks)
    #df_picks.to_csv("../../../Data/{p}/Picks/{q}_picks.csv".format(p=Creepmeter_dataframe.Network.iloc[i],\
    #                                                                q =Creepmeter_dataframe.File_code.iloc[i]))

In [ ]:
37 is CE